In [1]:
import pandas as pd
import ast
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load your CSV
df = pd.read_csv("dataset.csv")
df.head()


c:\Users\Rajveer Mathur\Desktop\NER-BERT-finetuning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Template,Filled Template,Tokenised Filled Template,Tokens
0,"In our video conference, discuss the role of e...","In our video conference, discuss the role of e...","['in', 'our', 'video', 'conference', ',', 'dis...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,Could you draft a letter for [NAME_1] to send ...,"Could you draft a letter for Dietrich, Schulis...","['could', 'you', 'draft', 'a', 'letter', 'for'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-NAME', 'I-NA..."
2,Discuss the options for [FULLNAME_1] who wants...,Discuss the options for Jeffery Pfeffer who wa...,"['discuss', 'the', 'options', 'for', 'jeff', '...","['O', 'O', 'O', 'O', 'B-FULLNAME', 'I-FULLNAME..."
3,13. Write a press release announcing [FULLNAME...,13. Write a press release announcing Gayle Wat...,"['13', '.', 'write', 'a', 'press', 'release', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FULLNAM..."
4,9. Develop an inventory management plan for [F...,9. Develop an inventory management plan for Ev...,"['9', '.', 'develop', 'an', 'inventory', 'mana...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FU..."


In [2]:
def try_parse_list(cell):
    if isinstance(cell, str):
        return ast.literal_eval(cell)
    return cell

df["Tokenised Filled Template"] = df["Tokenised Filled Template"].apply(try_parse_list)
df["Tokens"] = df["Tokens"].apply(try_parse_list)
df.head()

,Template,Filled Template,Tokenised Filled Template,Tokens
0,"In our video conference, discuss the role of e...","In our video conference, discuss the role of e...","[in, our, video, conference, ,, discuss, the, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,Could you draft a letter for [NAME_1] to send ...,"Could you draft a letter for Dietrich, Schulis...","[could, you, draft, a, letter, for, dietrich, ...","[O, O, O, O, O, O, B-NAME, I-NAME, I-NAME, I-N..."
2,Discuss the options for [FULLNAME_1] who wants...,Discuss the options for Jeffery Pfeffer who wa...,"[discuss, the, options, for, jeff, ##ery, p, #...","[O, O, O, O, B-FULLNAME, I-FULLNAME, I-FULLNAM..."
3,13. Write a press release announcing [FULLNAME...,13. Write a press release announcing Gayle Wat...,"[13, ., write, a, press, release, announcing, ...","[O, O, O, O, O, O, O, B-FULLNAME, I-FULLNAME, ..."
4,9. Develop an inventory management plan for [F...,9. Develop an inventory management plan for Ev...,"[9, ., develop, an, inventory, management, pla...","[O, O, O, O, O, O, O, O, B-FULLNAME, I-FULLNAM..."


In [3]:
# Check for mismatches between tokens and labels
counter = 0
for idx, row in df.iterrows():
    if len(row["Tokenised Filled Template"]) != len(row["Tokens"]):
        print(f"Mismatch at index {idx}")
        # print(f"Tokens ({len(row['Tokenised Filled Template'])}): {row['Tokenised Filled Template']}")
        # print(f"Labels ({len(row['Tokens'])}): {row['Tokens']}")
        counter += 1
print(f"Total mismatches found: {counter}")

Mismatch at index 701
Mismatch at index 1123
Mismatch at index 1906
Mismatch at index 2141
Mismatch at index 2192
Mismatch at index 2205
Mismatch at index 2375
Mismatch at index 2387
Mismatch at index 3218
Mismatch at index 3381
Mismatch at index 3654
Mismatch at index 3870
Mismatch at index 4265
Mismatch at index 4993
Mismatch at index 5071
Mismatch at index 5135
Mismatch at index 6312
Mismatch at index 6801
Mismatch at index 7190
Mismatch at index 7384
Mismatch at index 7779
Mismatch at index 7843
Mismatch at index 8652
Mismatch at index 8710
Mismatch at index 10344
Mismatch at index 10800
Mismatch at index 11358
Mismatch at index 12127
Mismatch at index 12358
Mismatch at index 13103
Mismatch at index 13467
Mismatch at index 13574
Mismatch at index 13642
Mismatch at index 13709
Mismatch at index 14074
Mismatch at index 14272
Mismatch at index 14604
Mismatch at index 15549
Mismatch at index 15775
Mismatch at index 15847
Mismatch at index 16239
Mismatch at index 16297
Mismatch at index

In [4]:
# Filter only rows where tokens and labels match in length
df = df[df["Tokenised Filled Template"].str.len() == df["Tokens"].str.len()]
# Check for mismatches between tokens and labels
counter = 0
for idx, row in df.iterrows():
    if len(row["Tokenised Filled Template"]) != len(row["Tokens"]):
        print(f"Mismatch at index {idx}")
        # print(f"Tokens ({len(row['Tokenised Filled Template'])}): {row['Tokenised Filled Template']}")
        # print(f"Labels ({len(row['Tokens'])}): {row['Tokens']}")
        counter += 1
print(f"Total mismatches found: {counter}")

Total mismatches found: 0


In [5]:
# Extract BIO tag scheme
all_tags = set(tag for tags in df["Tokens"] for tag in tags)
unique_tags = sorted(all_tags)
label2id = {tag: idx for idx, tag in enumerate(unique_tags)}
id2label = {idx: tag for tag, idx in label2id.items()}

# Add numeric label ids for each row
df["Label_ids"] = df["Tokens"].apply(lambda tags: [label2id[tag] for tag in tags])

# Train-validation split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face datasets
def to_hf_dataset(dataframe):
    return Dataset.from_dict({
        "tokens": dataframe["Tokenised Filled Template"].tolist(),
        "labels": dataframe["Label_ids"].tolist()
    })

train_dataset = to_hf_dataset(train_df)
val_dataset = to_hf_dataset(val_df)


In [6]:
from transformers import BertTokenizerFast, BertForTokenClassification

model_checkpoint = "bert-base-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

model = BertForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(unique_tags),
    id2label=id2label,
    label2id=label2id
)


W0803 00:33:54.709000 8620 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\Rajveer Mathur\Desktop\NER-BERT-finetuning\.venv\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",   # ensure uniform tensor sizes
        max_length=128,
        return_attention_mask=True
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # For subword tokens, assign -100 or the same label as the word
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 2294/2294 [00:00<00:00, 2545.82 examples/s]


In [11]:
from transformers import TrainingArguments, Trainer
import numpy as np
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(predictions_output):
    preds, labels = predictions_output
    preds = np.argmax(preds, axis=2)

    true_preds = []
    true_labels = []

    for pred_seq, label_seq in zip(preds, labels):
        pred_tags = []
        label_tags = []
        for p, l in zip(pred_seq, label_seq):
            if l != -100:
                pred_tags.append(id2label[p])
                label_tags.append(id2label[l])
        true_preds.append(pred_tags)
        true_labels.append(label_tags)

    return {
        "accuracy": accuracy_score(true_labels, true_preds),
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
    }

training_args = TrainingArguments(
    output_dir="./ner_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


                                         
  0%|          | 0/12905 [12:28<?, ?it/s]            

{'loss': 3.72, 'grad_norm': 11.179600715637207, 'learning_rate': 1.9984502130956996e-05, 'epoch': 0.0}


                                         
  0%|          | 0/12905 [13:29<?, ?it/s]            

{'loss': 1.6989, 'grad_norm': 2.8210830688476562, 'learning_rate': 1.9969004261913987e-05, 'epoch': 0.01}


                                         
  0%|          | 0/12905 [14:23<?, ?it/s]            

...

                                         
  0%|          | 0/12905 [32:10<?, ?it/s]             

{'loss': 0.4426, 'grad_norm': 3.7190933227539062, 'learning_rate': 1.9581557535838824e-05, 'epoch': 0.1}


                                         
  0%|          | 0/12905 [32:52<?, ?it/s]             

{'loss': 0.5278, 'grad_norm': 12.9356107711792, 'learning_rate': 1.956605966679582e-05, 'epoch': 0.11}


KeyboardInterrupt: 

In [ ]:
trainer.save_model("./final_ner_model")
tokenizer.save_pretrained("./final_ner_model")

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="./final_ner_model", tokenizer="./final_ner_model", aggregation_strategy="simple")

text = "Dr. Marvin Rolfson and Julius Daugherty attended the arbitration."
print(ner_pipeline(text))
